## <strong> <a style="color:black">LSTM Model Accuracy for all Stocks </a> </strong> 

---

|  |  |
|:---|:---|
|**Reading Time** |  15 minutes |
|**Packages used** | Defined with relevant DocStrings within repository   |
|**Models** | Read already tuned models saved in repository ./model/output/LSTM/|
|**Outcome** | Comparison of the model accuracy for 2 scenarios | 
| | 1) Daily prediction for next day stock price and comparing it with actual trend|
| | 2) Multi-step forecasting in future at time t=0 by using predicted values as inputs, and comparing it with actual trend|

---

#### In this notebook, I have demonstrated the performance of trained model for the set of below stocks: -

| **Stocks** |
| :---|
| State Bank of India |
| Reliance |
| Airtel |
| ONGC |
| Tech Mahindra |
| HUL |
| TCS |
| Asian Paints|
| Eicher Motors |
| Axis Bank |

---

#### For all the above stocks, I have built a LSTM model based on the below features to predict the next day price:-

| **Price History**| **Technical Indicators**|
| :---| :---|
| Last 10 day price history of Stock|  Simple Moving Average of Last 20 days|
|| Simple Moving Average of Last 50 days| 
|| Relative Strength Index (RSI)|
|| Moving Average Convergence Divergence|
|| Upper and lower Bollinger Band|

---

#### Plots

I have plottted 2 types of plots to demonstrate the performance of the model: - 


1. ) Predict next day stock price based on the updated new features by incorporating the efffect of new day data due to each passing day in the features.

2. )  Predict price based on multi-step forecasting technique where I have used the predicted values from the model as an input to predict the further next day price in future, thus, all future values of stock price are predicted at the time t=0.

---